## Tables

In this notebook I cover the methods behind the main result Tables (2, 3 & 4) for the univariate analysis. Most of the magic occurs inside `babble.Babbler.examinate` in the `MQDQParser` package, if you're into reading the code.


In [1]:
from bs4 import BeautifulSoup

import importlib

from mqdq import rhyme, utils, rhyme_classes, babble
from mqdq import line_analyzer as la

import random
import operator
import bisect
import string
import copy
import re
import scipy as sp
import pandas as pd
import glob
import umap
import os
import subprocess

### Parse the MQDQ data

The class that was designed to do the simulation ended up being called `Babbler`, although it now incorporates the analysis and a few other utility methods. One Babbler object represents a single text, and there are a variety of construction methods. Because the MQDQ files for the texts were a mix of single files and book-by-book files the invocation below is just done ad-hoc instead of with some overarching utility method.

In [2]:
allbabs = []

tris_bab = babble.Babbler.from_file(*sorted(glob.glob('mqdq/OV-tri*.xml')), name = 'Tristia')
allbabs.append(tris_bab)

tib_babs = babble.multibabs(sorted(glob.glob('mqdq/TIB-ele*.xml')),"Tibullus")
allbabs.extend(tib_babs)

tib_single_bab = babble.Babbler.from_file(*sorted(glob.glob('mqdq/TIB-ele*.xml')),name ="Tibullus")
allbabs.append(tib_single_bab)

cat_soup, cat_ll = utils.slurp('mqdq/CATVLL-carm.xml')
cat64_bab = babble.Babbler(utils.clean(cat_soup('division')[63]('line')), name = "Cat64")
allbabs.append(cat64_bab)

aen_babs = babble.bookbabs('mqdq/VERG-aene.xml', 'Aeneid')
allbabs.extend(aen_babs)

aen_single_bab = babble.Babbler.from_file('mqdq/VERG-aene.xml', name='Aeneid')
allbabs.append(aen_single_bab)

geo_babs = babble.bookbabs('mqdq/VERG-geor.xml', 'Georgics')
allbabs.extend(geo_babs)

geo_single_bab = babble.Babbler.from_file('mqdq/VERG-geor.xml', name='Georgics')
allbabs.append(geo_single_bab)

sat_babs = babble.bookbabs('mqdq/IVV-satu.xml', 'Juv. Sat.')
allbabs.extend(sat_babs)

sat_single_bab = babble.Babbler.from_file('mqdq/IVV-satu.xml', name='Juv. Sat.')
allbabs.append(sat_single_bab)

met_babs = babble.bookbabs('mqdq/OV-meta.xml', 'Metamorphoses')
allbabs.extend(met_babs)

met_single_bab = babble.Babbler.from_file('mqdq/OV-meta.xml', name='Metamorphoses')
allbabs.append(met_single_bab)

puni_babs = babble.bookbabs('mqdq/SIL-puni.xml', 'Punica')
allbabs.extend(puni_babs)

puni_single_bab = babble.Babbler.from_file('mqdq/SIL-puni.xml', name='Punica')
allbabs.append(puni_single_bab)

theb_babs = babble.bookbabs('mqdq/STAT-theb.xml', 'Thebaid')
allbabs.extend(theb_babs)

theb_single_bab = babble.Babbler.from_file('mqdq/STAT-theb.xml', name='Thebaid')
allbabs.append(theb_single_bab)

phars_babs = babble.bookbabs('mqdq/LVCAN-phar.xml', 'Pharsalia')
allbabs.extend(phars_babs)

phars_single_bab = babble.Babbler.from_file('mqdq/LVCAN-phar.xml', name='Pharsalia')
allbabs.append(phars_single_bab)

prop_babs = babble.multibabs(sorted(glob.glob('mqdq/PROP-ele*.xml')),"Propertius")
allbabs.extend(prop_babs)

prop_single_bab = babble.Babbler.from_file(*sorted(glob.glob('mqdq/PROP-ele*.xml')),name ="Propertius")
allbabs.append(prop_single_bab)

ep_bab = babble.Babbler.from_file('mqdq/OV-epis.xml', name="Heroides")
allbabs.append(ep_bab)

aram_bab = babble.Babbler.from_file('mqdq/OV-aram.xml', name="Ars")
allbabs.append(aram_bab)

fast_bab = babble.Babbler.from_file('mqdq/OV-fast.xml', name="Fasti")
allbabs.append(fast_bab)

arg_babs = babble.bookbabs('mqdq/VAL_FL-argo.xml', 'Argonautica')
allbabs.extend(arg_babs)

arg_single_bab = babble.Babbler.from_file('mqdq/VAL_FL-argo.xml', name='Argonautica')
allbabs.append(arg_single_bab)

rena_babs = babble.bookbabs('mqdq/LVCR-rena.xml', 'DRN')
allbabs.extend(rena_babs)

rena_single_bab = babble.Babbler.from_file('mqdq/LVCR-rena.xml', name='DRN')
allbabs.append(rena_single_bab)

apot_bab = babble.Babbler.from_file('mqdq/PRVD-apot.xml', name='Apotheosis')
allbabs.append(apot_bab)

hamart_bab = babble.Babbler.from_file('mqdq/PRVD-hama.xml', name='Hamartigenia')
allbabs.append(hamart_bab)

psych_bab = babble.Babbler.from_file('mqdq/PRVD-psyc.xml', name='Psychomachia')
allbabs.append(psych_bab)

horsat_babs = babble.multibabs(sorted(glob.glob('mqdq/HOR-sat*.xml')),"Hor. Sat.")
allbabs.extend(horsat_babs)

horsat_single_bab = babble.Babbler.from_file(*sorted(glob.glob('mqdq/HOR-sat*.xml')),name ="Hor. Sat.")
allbabs.append(horsat_single_bab)

### Converting a DataFrame to an observation vector

Converting a result df to a vector for multivariate analysis, as described in Section 4.2 (28 $\Pi$ scores, in this case the last 14 P- scores are the duplicated H- scores.

In [6]:
babble._pivot(apot_df)

,H-aa -1,H-aa -2,H-aa 0,H-aa mid,H-axa -1,H-axa -2,H-axa 0,H-axa mid,H-axxa -1,H-axxa -2,...,P-axa -1,P-axa -2,P-axa 0,P-axa mid,P-axxa -1,P-axxa -2,P-axxa 0,P-axxa mid,P-leo,P-slant leo
work,,,,,,,,,,,,,,,,,,,,,
Apotheosis,1.075758,1.204545,1.444444,1.264151,1.042254,1.235955,1.235294,1.017544,1.0,0.98913,...,1.042254,1.235955,1.235294,1.017544,1.0,0.98913,1.264706,1.283019,1.636364,1.27027


### Run the analysis on every text [WARNING: 10+ HOURS TO RUN]

This is the main analysis method. It takes a long time to run. There are 174k lines in the allbabs array (most works appear twice--once individually and once broken into books) and each line is simulated 101 times so we're simulating about 17 million lines here. I did quite a bit of work on optimisation, but more is surely possible, and would be required to do _really_ big corpus work.

See below for a cell which just imports the pickled version of the results, generated using this random seed.

In [56]:
# random.seed(42)
# megadf = [b.examinate(tests=babble.extended_tests) for b in allbabs]

In [57]:
# full_results = pd.concat(megadf,ignore_index=True)
# full_results

## Corpus Size

19 works, 12 authors, 174k lines in total, but only 96k unique lines (works are included in full and also in their individual books for larger works)

In [9]:
full_works = [x for x in allbabs if not re.search('[0-9]', x.name)]

In [10]:
sum([len(x.raw_source) for x in allbabs])

174569

In [11]:
sum([len(x.raw_source) for x in full_works])

95956

In [492]:
len(full_works)

19

In [493]:
[x.name for x in full_works]

['Tristia',
 'Tibullus',
 'Aeneid',
 'Georgics',
 'Juv. Sat.',
 'Metamorphoses',
 'Punica',
 'Thebiad',
 'Pharsalia',
 'Propertius',
 'Heroides',
 'Ars',
 'Fasti',
 'Argonautica',
 'DRN',
 'Apotheosis',
 'Hamartigenia',
 'Psychomachia',
 'Hor. Sat.']

## Baselines

Every Babbler eventually needs to calculate its _baselines_, as described in Sections 3.4 and 3.5. The results are calculated by metre and also by position, so in the baselines for the _Aeneid_ below the only metre is 'H' for hexameter. For large texts, there will be three figures - a median simulated baseline and a 5% and 95% simulated value, based on 100 x 50,000 simulated lines. For the _Aeneid_ in this example, the median for end-rhymes (-1) is 0.07382, which is to say that a random pair of lines should be expected to rhyme in the final word 7.38% of the time. Very small texts will not have the 5% and 95% values because the probabilities can be calculated precisely (every word from the positional set is compared with every other word to give an empirical chance that word-pairs rhyme).

In [450]:
aen_single_bab.baselines()

{'H': {-1: (0.07128, 0.07382, 0.07516),
  -2: (0.09344, 0.09568, 0.09766),
  'mid': (0.038127648340934635, 0.039571316579086524, 0.04131112189173112),
  'leo': (0.044597943089430894, 0.04667584552845529, 0.04867678861788618)}}

"cross" results are pairs where one line is a hexameter and one is a pentameter. This is relevant to many of the vertical rhymes, and is discussed in Section 4.1.1.

In [46]:
fast_bab.baselines()

{'H': {-1: (None, 0.07802405745818435, None),
  -2: (None, 0.09856079356266319, None),
  'mid': (None, 0.05040867107855935, None),
  'leo': (0.05333555913113436, 0.055393073209975864, 0.057570442477876106)},
 'P': {-1: (None, 0.09301148807567852, None),
  -2: (None, 0.1570488093484479, None),
  'mid': (None, 0.06410498388561457, None),
  'leo': (0.06126, 0.06354, 0.06574)},
 'cross': {-1: (0.06846, 0.07042, 0.0729),
  -2: (0.098, 0.10092, 0.10324),
  'mid': (0.05155770716009654, 0.053175752212389384, 0.0547738213998391)}}

## Pickling/Unpickling

If you don't want to wait for many hours, you can just use the pickled version of the results, based on the random seed above.

In [9]:
full_results.to_pickle('full_results_42.pkl')

In [58]:
full_results = pd.read_pickle('full_results_42.pkl')
full_results

,work,metre,test,pi,stars,t,f,sum,expected,alternative,binom,conservative,l99,l95,l90,mid,h90,h95,h99
0,Tristia,H,leo,1.423913,***,131,1635,1766,91,greater,0.000033,0.000033,68,77,79,92,108,116,120
1,Tristia,H,slant leo,1.057971,,73,1693,1766,0,None,1.000000,1.000000,48,56,59,69,83,86,94
2,Tristia,H,aa 0,1.423729,***,84,1681,1765,58,greater,0.000630,0.000630,42,43,45,59,71,73,74
3,Tristia,H,axa 0,1.539683,***,97,1668,1765,61,greater,0.000007,0.000007,45,49,51,63,76,78,86
4,Tristia,H,axxa 0,1.203390,*,71,1693,1764,58,greater,0.049878,0.049878,39,45,46,59,69,71,73
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1955,Hor. Sat.,H,axa -2,0.898734,,142,1968,2110,150,less,0.254738,0.254738,131,134,136,158,179,184,201
1956,Hor. Sat.,H,axxa -2,1.019108,,160,1949,2109,150,greater,0.216765,0.216765,134,136,137,157,170,182,199
1957,Hor. Sat.,H,aa mid,1.094737,,104,2007,2111,93,greater,0.132339,0.132339,71,79,82,95,108,110,112
1958,Hor. Sat.,H,axa mid,1.053191,,99,2011,2110,93,greater,0.273025,0.273025,71,76,79,94,110,111,113


## Table 2 - an example analysis

Table 2 is mostly a LaTeX version of the pandas.DataFrame output of examinate.

Some columns are omitted in the paper Table. 't', 'f' and 'sum' are the counts (true or false) for the given test. The binomial test works off the Bernoulli probabilities (Section 3.5) which are calculated exactly for smaller tests (so the conservative P is identical to the normal P), and using a simulated confidence margin for larger tests. The 'conservative' column is a binomial P-value calculated using the h95 or l95 value (depending whether the test alternative is greater or lesser) instead of using the median ('mid'). This gives a much more conservative estimate. The 90th and 99th percentiles are also dropped to give just the most common 5, 50, 95 figures.

Also, which maybe useful to someone (it was life-changing for me) -- pandas has native `to_latex` for DataFrames. The output is not completely perfect but it's a great start!

In [55]:
apot_df = full_results[full_results.work=='Apotheosis']
print(apot_df.drop(['f', 'sum', 'l99', 'l90', 'h90', 'h99'],axis=1).to_latex(float_format="%.4f", index=False))

\begin{tabular}{lllrlrrlrrrrr}
\toprule
       work & metre &       test &     pi & stars &    t &  expected & alternative &  binom &  conservative &  l95 &  mid &  h95 \\
\midrule
 Apotheosis &     H &        leo & 1.6667 &   *** &   90 &        53 &     greater & 0.0000 &        0.0000 &   40 &   54 &   69 \\
 Apotheosis &     H &  slant leo & 1.2368 &       &   47 &         0 &        None & 1.0000 &        1.0000 &   28 &   38 &   50 \\
 Apotheosis &     H &       aa 0 & 1.4857 &    ** &   52 &        35 &     greater & 0.0038 &        0.0038 &   27 &   35 &   48 \\
 Apotheosis &     H &      axa 0 & 1.2000 &       &   42 &        35 &     greater & 0.1341 &        0.1341 &   23 &   35 &   46 \\
 Apotheosis &     H &     axxa 0 & 1.2286 &       &   43 &        35 &     greater & 0.1012 &        0.1012 &   25 &   35 &   48 \\
 Apotheosis &     H &      aa -1 & 1.0143 &       &   71 &        69 &     greater & 0.4161 &        0.4161 &   52 &   70 &   88 \\
 Apotheosis &     H &     a

## Table 3 - All but Leonine rhymes

In [48]:
# Take any work which
# - does not contain 'leo' in the test name (leo and slant leo)
# - does not have a book number in the Work name (ie only full works)
# - has at least one star (only statistically significant results

all_but_leo = full_results[
    (~full_results['test'].str.contains('leo')) & 
    (~full_results['work'].str.contains('[0-9]')) & 
    (full_results['stars'].str.contains('\*'))
].sort_values(by=['work', 'test'], ascending=True)
all_but_leo

,work,metre,test,pi,stars,t,f,sum,expected,alternative,binom,conservative,l99,l95,l90,mid,h90,h95,h99
299,Aeneid,H,aa -1,1.081006,**,774,9064,9838,725,greater,0.032285,0.136696,659,669,677,716,758,766,775
301,Aeneid,H,axxa -1,1.075967,**,779,9057,9836,725,greater,0.020529,0.098649,666,680,683,724,767,770,779
1884,Apotheosis,H,aa -2,1.139785,*,106,976,1082,90,greater,0.043281,0.043281,63,73,77,93,105,107,112
1878,Apotheosis,H,aa 0,1.485714,**,52,1030,1082,35,greater,0.003812,0.003812,24,27,28,35,45,48,57
1885,Apotheosis,H,axa -2,1.222222,*,110,971,1081,90,greater,0.016150,0.016150,72,74,76,90,109,110,112
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2,Tristia,H,aa 0,1.423729,***,84,1681,1765,58,greater,0.000630,0.000630,42,43,45,59,71,73,74
3,Tristia,H,axa 0,1.539683,***,97,1668,1765,61,greater,0.000007,0.000007,45,49,51,63,76,78,86
17,Tristia,P,axa 0,1.287879,**,85,1679,1764,64,greater,0.006767,0.006767,48,50,52,66,77,82,92
10,Tristia,H,axxa -2,1.141361,*,218,1546,1764,188,greater,0.013874,0.013874,159,166,171,191,211,218,225


In [50]:
print(all_but_leo.drop(['f', 'sum', 'l99', 'l90', 'h90', 'h99', 'conservative'],axis=1).to_latex(
    float_format="%.4f",
    column_format=r"lclcccclcc@{\hspace{1\tabcolsep}}c@{\hspace{1\tabcolsep}}c",
    index=False
))

\begin{tabular}{lclcccclcc@{\hspace{1\tabcolsep}}c@{\hspace{1\tabcolsep}}c}
\toprule
          work & metre &      test &     pi & stars &     t &  expected & alternative &  binom &   l95 &   mid &   h95 \\
\midrule
        Aeneid &     H &     aa -1 & 1.0810 &    ** &   774 &       725 &     greater & 0.0323 &   669 &   716 &   766 \\
        Aeneid &     H &   axxa -1 & 1.0760 &    ** &   779 &       725 &     greater & 0.0205 &   680 &   724 &   770 \\
    Apotheosis &     H &     aa -2 & 1.1398 &     * &   106 &        90 &     greater & 0.0433 &    73 &    93 &   107 \\
    Apotheosis &     H &      aa 0 & 1.4857 &    ** &    52 &        35 &     greater & 0.0038 &    27 &    35 &    48 \\
    Apotheosis &     H &    axa -2 & 1.2222 &     * &   110 &        90 &     greater & 0.0161 &    74 &    90 &   110 \\
   Argonautica &     H &  axxa mid & 0.8878 &     * &   269 &       303 &        less & 0.0209 &   267 &   303 &   335 \\
           Ars &     H &      aa 0 & 2.2632 &   *** 

## Table 4 - Leonine rhymes

In [49]:
# Similar to the above, take the results for leo and slant leo, only take full works,
# but (because there are so few works which do not show a significant propensity,
# don't worry about whether or not they have stars

leo_results = full_results[
    (full_results['test'].str.contains('leo')) & 
    (~full_results['work'].str.contains('[0-9]'))# & 
    #(full_results['stars'].str.contains('\*'))
].sort_values(by=['work', 'test'], ascending=True)
leo_results

,work,metre,test,pi,stars,t,f,sum,expected,alternative,binom,conservative,l99,l95,l90,mid,h90,h95,h99
294,Aeneid,H,leo,1.196078,***,549,9290,9839,459,greater,1.524895e-05,4.458460e-04,409,420,424,459,492,504,510
295,Aeneid,H,slant leo,1.109145,,376,9463,9839,0,None,1.000000e+00,1.000000e+00,304,309,313,339,376,380,399
1876,Apotheosis,H,leo,1.666667,***,90,993,1083,53,greater,1.609639e-06,1.609639e-06,37,40,43,54,66,69,71
1877,Apotheosis,H,slant leo,1.236842,,47,1036,1083,0,None,1.000000e+00,1.000000e+00,27,28,28,38,48,50,58
1764,Argonautica,H,leo,1.681672,***,523,5037,5560,310,greater,3.755518e-30,7.970632e-28,266,282,290,311,340,343,356
1765,Argonautica,H,slant leo,1.348739,***,321,5239,5560,0,None,1.000000e+00,1.000000e+00,208,211,217,238,268,274,278
1596,Ars,H,leo,1.676471,***,114,1051,1165,67,greater,5.713510e-08,5.713510e-08,47,54,56,68,81,84,85
1610,Ars,P,leo,2.041096,***,149,1015,1164,71,greater,2.731984e-17,2.731984e-17,55,57,58,73,86,86,91
1597,Ars,H,slant leo,1.615385,***,84,1081,1165,0,None,1.000000e+00,1.000000e+00,39,40,41,52,62,65,69
1611,Ars,P,slant leo,1.418182,***,78,1086,1164,0,None,1.000000e+00,1.000000e+00,41,43,44,55,67,70,77


In [52]:
print(leo_results.drop(['f', 'sum', 'l99', 'l90', 'h90', 'h99', 'conservative'],axis=1).to_latex(
    float_format="%.4f",
    column_format=r"lclcccclcc@{\hspace{1\tabcolsep}}c@{\hspace{1\tabcolsep}}c",
    index=False
))

\begin{tabular}{lclcccclcc@{\hspace{1\tabcolsep}}c@{\hspace{1\tabcolsep}}c}
\toprule
          work & metre &       test &     pi & stars &    t &  expected & alternative &  binom &  l95 &  mid &  h95 \\
\midrule
        Aeneid &     H &        leo & 1.1961 &   *** &  549 &       459 &     greater & 0.0000 &  420 &  459 &  504 \\
        Aeneid &     H &  slant leo & 1.1091 &       &  376 &         0 &        None & 1.0000 &  309 &  339 &  380 \\
    Apotheosis &     H &        leo & 1.6667 &   *** &   90 &        53 &     greater & 0.0000 &   40 &   54 &   69 \\
    Apotheosis &     H &  slant leo & 1.2368 &       &   47 &         0 &        None & 1.0000 &   28 &   38 &   50 \\
   Argonautica &     H &        leo & 1.6817 &   *** &  523 &       310 &     greater & 0.0000 &  282 &  311 &  343 \\
   Argonautica &     H &  slant leo & 1.3487 &   *** &  321 &         0 &        None & 1.0000 &  211 &  238 &  274 \\
           Ars &     H &        leo & 1.6765 &   *** &  114 &        67 &